In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.optimize import minimize

def my_logistic_regression(df_competition):
    """
    Computes
    """

    # format data
    data = df_competition.values  # convert data frame to numpy array
    m, n = data[:, :-1].shape
    X = np.concatenate((np.ones((m, 1)), data[:, :-1]), axis=1)
    y = data[:, -1]

    # optimize theta
    initial_theta = np.zeros(X.shape[1])
    res = minimize(costFunction, initial_theta, args=(X, y),
               method=None, jac=gradient, options={'maxiter': 1000})
    return res.x


def sigmoid(z):
    """
    Computes the sigmoid function for all values of z.

    :param z: scalar, vector or array of values for which the sigmoid function should be computed
    :returns: scalar, vector or array of values of size z containing computed sigmoid values
    """
    return (1 / (1 + np.power(np.e, -z)))


def costFunction(theta, X, y):
    """
    Computes the cost of using theta as the parameter for logistic regression and the
    gradient of the cost w.r.t. to the parameters

    :param theta: parameter vector containing corresponding values of theta
    :param X: MxN array that contains feature set (x-values)
    :param y: Mx1 array that contains resulting outcomes (y-values)
    :returns: a scalar of the cost "J" and gradient "grad" of the cost with the same size as theta
    """
    m = len(y)
    h = sigmoid(np.dot(X, theta))  # get predictions
    J = -(1 / m) * (np.log(h).T.dot(y) +
                    np.log(1 - h).T.dot(1 - y))  # calculate cost

    return J


def gradient(theta, X, y):
    """
    Computes the gradient of the cost with respect to cost J and theta.

    :param theta: parameter vector containing corresponding values of theta
    :param X: MxN array that contains feature set (x-values)
    :param y: Mx1 array that contains resulting outcomes (y-values)
    :returns: gradient of the cost with the same size as theta
    """
    m = len(y)
    h = sigmoid(np.dot(X, theta))
    return (1 / m) * np.dot(X.transpose(), h - y)


In [ ]:
df_competition_evaluation = pd.read_csv('C:/Users/802161/Source/github/glm-competition/data/df_competition.csv')

# listed of average AUCs
AUC_avgs = []

for avg in range(0, 10):
    # list of calculated AUCs
    AUC_list = []
    
    for iteration in range(0, 100):
        n = len(df_competition_evaluation)

        # divide into training and test
        l = range(0, n)
        k = int(0.6 * (n))
        training_index = random.sample(l, k)
        df_training = df_competition_evaluation.iloc[training_index]
        test_index = [x for x in l if x not in training_index]
        df_test = df_competition_evaluation.iloc[test_index]
        glm_results = my_logistic_regression(df_training)

        # calculate the raw predictions on the test set
        a0 = glm_results[0]
        a1 = glm_results[1]
        a2 = glm_results[2]
        a3 = glm_results[3]

        glm_pred = 1 / \
            (1 + np.exp(-(a0 + a1 * df_test['V1'] +
             a2 * df_test['V2'] + a3 * df_test['V3'])))

        # calculate the ROC
        TPR = []
        FPR = []
        thresholds = np.arange(0, 1.001, 0.001)
        m = len(thresholds)
        pd.options.mode.chained_assignment = None  # default='warn'
        df_test['glm_pred'] = glm_pred

        for threshold in thresholds:
            TP = len(df_test[(df_test['Target'] == 1) &
                             (df_test['glm_pred'] > threshold)])
            FN = len(df_test[(df_test['Target'] == 1) &
                             (df_test['glm_pred'] < threshold)])
            FP = len(df_test[(df_test['Target'] == 0) &
                             (df_test['glm_pred'] > threshold)])
            TN = len(df_test[(df_test['Target'] == 0) &
                             (df_test['glm_pred'] < threshold)])
            TPR.append(TP / (TP + FN))
            FPR.append(FP / (FP + TN))

        TPR[0] = 1
        FPR[0] = 1
        TPR[m - 1] = 0
        FPR[m - 1] = 0

        # Integrating the ROC to get the AUC
        c = np.array([sum(n) / 2 for n in zip(*[TPR[0:(m - 1)], TPR[1:m]])])
        AUC = abs(round(sum(np.diff(FPR) * c), 3))
        AUC_list.append(AUC)

    AUC_avgs.append(np.mean(AUC_list))